In [1]:
PROJECT_FOLDER = "/apps/prod/logos"

In [2]:
# # For future reference, this is how we can set up the test database in a notebook, for examples

# import sys, os
# DJANGO_LOCATION = "/Users/pvankessel/.pyenv/versions/3.6.5/envs/python3/lib/python3.6/site-packages"
# sys.path.append(DJANGO_LOCATION)
# import django
# os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# os.environ['DJANGO_SETTINGS_MODULE'] = 'testapp.settings'
# django.setup()

# from django import test
# from django.db import connection
# test.utils.setup_test_environment() # Setup the environment
# db = connection.creation.create_test_db() # Create the test db

# from testapp.tests.abstract_models import AbstractModelTests
# AbstractModelTests().setUp()

In [3]:
# Something we should probably add to django_pewtils
def set_up_django_project(project_name, project_path, env_file=None):
    
    import django
    import os, sys
    import numpy as np
    from contextlib import closing

    from rasterio.env import GDALDataFinder
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    os.environ["DJANGO_SETTINGS_MODULE"] = "{}.settings".format(project_name)
    os.environ["GDAL_DATA"] = GDALDataFinder().search()
    
    if env_file:
        with closing(open(env_file, "r")) as infile:
            for line in infile.readlines():
                key = line.split("=")[0]
                if key != "PYTHONPATH":
                    value = "=".join(line.split("=")[1:]).strip('"').strip("\n")
                    os.environ[key] = value

    for i, path in enumerate({project_path, '{}/src'.format(project_path)}):
        try:
            del sys.path[sys.path.index(path)]
        except ValueError:
            pass
        sys.path.insert(i, path)
    for folder in os.listdir('{}/src'.format(project_path)):
        sys.path.insert(0, os.path.join(project_path, "src", folder))
    if "/apps/prod" not in project_path:
        for path in list(sys.path):
            if '/apps/prod' in path:
                del sys.path[sys.path.index(path)]
    
    django.setup()
    
# Change the folder paths to wherever you have sermonator checked out
set_up_django_project(
    "logos", 
    PROJECT_FOLDER, 
    env_file="{}/deploy/prod/master/conf/logos.env".format(PROJECT_FOLDER)
)

/home/pvankessel/.local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from logos.models import *

# Basic functions

In [5]:
from django_pewtils import get_model

In [6]:
get_model("facebook page")

logos.models.facebook.FacebookPage

In [7]:
get_model("facebookpage")

logos.models.facebook.FacebookPage

In [8]:
get_model("Facebook_Page")

logos.models.facebook.FacebookPage

In [9]:
get_model("contenttype")

django.contrib.contenttypes.models.ContentType

In [10]:
get_model("document")

django_learning.models.documents.Document

In [11]:
from django_pewtils import reset_django_connection
reset_django_connection(app_name="logos")

In [12]:
from django_pewtils import get_all_field_names
get_all_field_names(Politician)

['education_bachelors',
 'age',
 'votes_for',
 'staffers',
 'education_bachelors_institution',
 'twitter_profiles',
 'webpages',
 'votes_against',
 'military_service_branch',
 'press_releases',
 'facebook_pages',
 'ballotpedia_id',
 'has_press_release_scraper',
 'command_logs',
 'last_name',
 'education_associates',
 'education_md_institution',
 'contributions_donated',
 'ranking_member_committees',
 'military_service',
 'committees',
 'elections_won',
 'current_term_id',
 'cosponsored_bills',
 'current_term',
 'caucuses',
 'valid_wikipedia_id',
 'fec_ids',
 'education_jd_institution',
 'suffix',
 'image_src',
 'latest_term_id',
 'education_masters_institution',
 'profession',
 'education_jd',
 'military_service_years',
 'ballotpedia_page',
 'nickname',
 'icpsr_id',
 'chaired_committees',
 'personal_metrics',
 'birthday',
 'old_facebook_ids',
 'middle_initial',
 'twitter_ids',
 'incumbent_elections',
 'lis_id',
 'education_associates_institution',
 'in_office',
 'id',
 'campaigns',
 'w

# The core of Django Pewtils: the `BasicExtendedModel` and `BasicExtendedManager`

Django Pewtils' main purpose is to extend the Django ORM with useful functions for working with records and queries in your database.  The overwhelming majority of these functions can be found on two classes.  The `BasicExtendedModel` extends Django's base Model class with additional row-level functions, and the `BasicExtendedManager` extends Django's base Manager class with additional table/query-level functions.  To use these extended classes, just swap out `models.Model` for `BasicExtendedModel` in your model definition.  It uses `BasicExtendedManager` by default, so you don't even have to worry about that.

```python
    class TestModel(BasicExtendedModel):
        pass
```

Let's start by taking a look at the Politician table, which contains members of Congress, presidential candidates, and other politicians

In [13]:
politicians = Politician.objects.all()

In [14]:
politicians

<PoliticianManager [<Politician: John Michael Fleig>, <Politician: James Berryhill>, <Politician: Joseph C Miechowicz>, <Politician: James Edgar Sr Md Lundeen>, <Politician: Daniel Cochcran 'Dc' Morrison>, <Politician: Mary Pallant>, <Politician: Charles Taylor Sutherland>, <Politician: Demetrios S Giannaros>, <Politician: Corinne Nicole Westerfield>, <Politician: Joseph M Kyrillos Jr>, <Politician: Paul Andrew Rundquist>, <Politician: Jim Bussler>, <Politician: Sona Mehring>, <Politician: William G. Barnes>, <Politician: Carol Ann Joyce Larosa>, <Politician: John R. Cox>, <Politician: >, <Politician: Go Vegan Go Vegan>, <Politician: Trish Causey>, <Politician: Christopher Alen Andrade>, '...(remaining elements truncated)...']>

In [15]:
politicians.count()

20673

If we wanted to quickly grab all of this data and start crunching numbers, we can do that easily by using the `.to_df` function provided by the `BasicExtendedManager`, which converts any arbitrary Django query into a Pandas DataFrame.  Just watch your memory with big queries!

In [16]:
politicians.to_df()

,id,first_name,middle_initial,last_name,nickname,suffix,has_press_release_scraper,religion,gender,birthday,...,birthplace,military_service,military_service_years,military_service_branch,profession,bioguide_bio,current_term_id,latest_term_id,party_id,in_office
0,49974,John,Michael,Fleig,,,False,None,None,NaT,...,None,None,NaN,None,None,None,NaN,NaN,NaN,None
1,62869,James,,Berryhill,,,False,None,None,NaT,...,None,None,NaN,None,None,None,NaN,NaN,NaN,None
2,49987,Joseph,C,Miechowicz,,,False,None,None,NaT,...,None,None,NaN,None,None,None,NaN,NaN,NaN,None
3,50087,James,Edgar Sr Md,Lundeen,,,False,None,None,NaT,...,None,None,NaN,None,None,None,NaN,NaN,NaN,None
4,50108,Daniel,Cochcran,Morrison,Dc,,False,None,None,NaT,...,None,None,NaN,None,None,None,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20668,58883,Julio,,Castaneda,,,False,None,None,NaT,...,None,None,NaN,None,None,None,NaN,NaN,NaN,None
20669,58884,Samir,,Jammal,,,False,None,None,NaT,...,None,None,NaN,None,None,None,NaN,NaN,NaN,None
20670,58911,Brian,,Forde,,,False,None,None,NaT,...,None,None,NaN,None,None,None,NaN,NaN,NaN,None
20671,58657,Joshua,A,Mandel,,,False,None,None,NaT,...,None,None,NaN,None,None,None,NaN,NaN,NaN,None


We could also pull a random sample using `.sample`

In [17]:
politicians.sample(10)

<PoliticianManager [<Politician: Gene Eugene Green>, <Politician: Michael C Hight>, <Politician: Jason Kander>, <Politician: Steven Edward Mirabella>, <Politician: Harold L Whitfield>, <Politician: Sheirl Lee Fletcher>, <Politician: Joe Manchik>, <Politician: Charles Wayne Dowdy>, <Politician: Nicholas Tutora>, <Politician: Matthew Caroll Hook>]>

In [18]:
politicians.sample(10).to_df()

,id,first_name,middle_initial,last_name,nickname,suffix,has_press_release_scraper,religion,gender,birthday,...,birthplace,military_service,military_service_years,military_service_branch,profession,bioguide_bio,current_term_id,latest_term_id,party_id,in_office
0,5804,Eric,J. J.,Massa,,,False,None,M,1959-09-16,...,None,None,None,None,None,"MASSA, Eric J.J., a Representative from New Yo...",None,9827.0,26.0,False
1,45570,Peter,,Vivaldi,,,False,None,None,NaT,...,None,None,None,None,None,None,None,NaN,NaN,None
2,47758,James,A,Hayden,,,False,None,None,NaT,...,None,None,None,None,None,None,None,NaN,NaN,None
3,51324,Clinton,,Desjarlais,,,False,None,None,NaT,...,None,None,None,None,None,None,None,NaN,NaN,None
4,54965,Adam,D,Shaffer,,,False,None,None,NaT,...,None,None,None,None,None,None,None,NaN,NaN,None
5,55248,Myrtle,Charlotte Montomery,Carlyle,,,False,None,None,NaT,...,None,None,None,None,None,None,None,NaN,NaN,None
6,57725,Billy,,Falling,,,False,None,None,NaT,...,None,None,None,None,None,None,None,NaN,NaN,None
7,59130,Angie,,Chirino,,,False,None,None,NaT,...,None,None,None,None,None,None,None,NaN,NaN,None
8,61854,Wednesday,Alexandra,Green,,,False,None,None,NaT,...,None,None,None,None,None,None,None,NaN,NaN,None
9,64080,Chris,B.,Royal,,,False,None,None,NaT,...,None,None,None,None,None,None,None,NaN,NaN,None


The Politician table isn't _super_ large, but if it was, fully evaluating the query could cause you problems - by default, Django will try to load queries into memory, even if you're just trying to loop over each record and do something with it one at a time.  To help with this, the `.chunk` function will efficiently load the full list of primary keys in your query and iterate over them in chunks, to keep things light.

In [19]:
for obj in politicians.chunk(size=1000):  # Behind the scenes, Django Pewtils will iterate over the records 1000 at a time
    pass  # do something

Similarly, if we want to make bulk changes to a set of records in a table, we can use the `chunk_update` function.  And if your query stalls when trying to delete records in bulk (like `politicians.delete()`), then the `chunk_delete` function can help you delete your records en masse.  (For obvious reasons, we won't be doing a live example of these here.)
    
```python
    politicians.chunk_update(first_name="Bob")
```

```python
    politicians.chunk_delete()
```

Anyway, let's explore some of the other Django Pewtils functions with an example.  The records that we see in our Politician table didn't come from nowhere - we had to compile our database from a variety of different data sources.  The @unitedstates GitHub is a great place to start - it has tons of information on members of Congress, including their names, terms of office, social media accounts, and more.  But there are tons of other sources too: Wikipedia has extensive bios on Congress, the FEC provides detailed campaign finance data, etc.  To bring all of this data together, we need to harmonize records from these various sources - which can be difficult because different sources use different unique identifiers, and not all of the data is perfectly clean.  

Let's see how Django Pewtils can help us with some of these challenges.  Let's imagine that Dwayne "The Rock" Johnson decides to run for President in 2024, but drops out of the race after losing to Oprah in the primaries, runs for Senate, gets elected there instead.  He first shows up in our database via the record below:

In [20]:
initial_record = {
    "bioguide_id": "J99999",
    "first_name": "Dwayne",
    "last_name": "Johnson"
}
Politician.objects.create(**initial_record)
Politician.objects.get(bioguide_id="J99999")

<Politician: Dwayne Johnson>

Now let's say we download data from another source that has some additional information on politicians - including the FEC ID for Senator Rock's failed presidential bid, and the ID for his campaign's Instagram account.

In [21]:
new_record = {
    "bioguide_id": "J99999",
    "fec_ids": ["P99999"],
    "last_name": "Johnson",
    "nickname": "The Rock",
    "instagram_ids": ["1234567890"]
}

Since we've been good database architects and we've specified that `bioguide_id` is a unique field, if we try to create a new record for Mr. Rock, it's going to fail because our first record already exists.

In [22]:
Politician.objects.create(**new_record)

IntegrityError: duplicate key value violates unique constraint "logos_politician_bioguide_id_317c4279_uniq"
DETAIL:  Key (bioguide_id)=(J99999) already exists.


So, traditionally, we'd write some code to catch the error, and if he already exists, we update the existing record instead

In [23]:
from django.db import IntegrityError

try: 
    Politician.objects.create(**new_record)
except IntegrityError:
    pol = Politician.objects.get(bioguide_id=new_record['bioguide_id'])
    for fec_id in new_record["fec_ids"]:
        if fec_id not in pol.fec_ids: 
            pol.fec_ids.append(fec_id)
    if not pol.last_name:
        pol.last_name = new_record['last_name']
    # ... and so on, and then we save the existing record:
    # pol.save() 

We could even be fancier and skip that IntegrityError check

In [24]:
pol, created = Politician.objects.get_or_create(bioguide_id=new_record['bioguide_id'])
for fec_id in new_record["fec_ids"]:
    if fec_id not in pol.fec_ids: 
        pol.fec_ids.append(fec_id)
if not pol.last_name:
    pol.last_name = new_record['last_name']
# ... and so on, and then we save the existing record:
# pol.save() 

But this is a royal pain.  Often, when we're trying to harmonize data from multiple sources, we're A) working with overlapping but incomplete records and multiple potential IDs, and B) working with data that can be easily represented as JSON/dictionary records. So wouldn't it be nice if we could just query Django directly with those records, and have it search for existing records across multiple fields?

In [25]:
new_record

{'bioguide_id': 'J99999',
 'fec_ids': ['P99999'],
 'last_name': 'Johnson',
 'nickname': 'The Rock',
 'instagram_ids': ['1234567890']}

In [26]:
pol = Politician.objects.get_if_exists(
    {"bioguide_id": new_record["bioguide_id"], "fec_ids": new_record['fec_ids']},
    match_any=True, search_nulls=False, empty_lists_are_null=True, allow_list_overlaps=False
)

The `BasicExtendedModel` `.json` function can fetch a dictionary representation of a particular record (the same way `.values()` does for all of the objects in your query in vanilla Django).

In [27]:
pol.json(exclude_nulls=True)

{'id': 64716,
 'first_name': 'Dwayne',
 'last_name': 'Johnson',
 'has_press_release_scraper': False,
 'bioguide_id': 'J99999',
 'fec_ids': [],
 'facebook_ids': [],
 'old_facebook_ids': [],
 'twitter_ids': [],
 'old_twitter_ids': [],
 'instagram_ids': [],
 'old_instagram_ids': [],
 'capitol_words_speech_backfill': False}

Sweet, we found a match.  Now, what if we could give Django some guidelines, pass it our identifiers AND our new data, and have it intelligently create or update records all at once?

In [28]:
pol = Politician.objects.create_or_update(
    {"bioguide_id": new_record["bioguide_id"], "fec_ids": new_record["fec_ids"]},
    new_record,
    match_any=True, search_nulls=False, empty_lists_are_null=True, allow_list_overlaps=True,
    save_nulls=False, only_update_existing_nulls=False, return_object=True
)

In [29]:
pol.json(exclude_nulls=True)

{'id': 64716,
 'first_name': 'Dwayne',
 'last_name': 'Johnson',
 'nickname': 'The Rock',
 'has_press_release_scraper': False,
 'bioguide_id': 'J99999',
 'fec_ids': ['P99999'],
 'facebook_ids': [],
 'old_facebook_ids': [],
 'twitter_ids': [],
 'old_twitter_ids': [],
 'instagram_ids': ['1234567890'],
 'old_instagram_ids': [],
 'capitol_words_speech_backfill': False}

Depending on the quality and completeness of our data source, we may have a preference for preserving any existing data, or alternatively overwriting it.  We can control this behavior with some of those keyword parameters - `save_nulls` (off by default) instructs django_pewtils to preserve non-null values that are null in our new data; `empty_lists_are_null` (on by default) determines whether empty lists should be treated like null values, and `only_update_existing_nulls` (off by default) is handy if you want to favor existing data and only want to fill in what's missing in any existing records.

Now let's say we collect yet another new record.  This time we don't have a Bioguide ID, but we do have FEC IDs - in fact, this new data source has both Mr. Rock's presidential FEC ID (which we know about) as well as his Senate race FEC ID (which we don't).  Let's pass all of the unique identifiers we have and see what happens.

In [30]:
new_record = {
    "icpsr_id": "12345",
    "fec_ids": ["S99999", "P99999"],
    "first_name": "Dwayne",
    "last_name": "Johnson",
    "nickname": None,
}

In [31]:
Politician.objects.get_if_exists(
    {"icpsr_id": new_record["icpsr_id"], "fec_ids": new_record["fec_ids"]},
    match_any=True, search_nulls=False, empty_lists_are_null=True, allow_list_overlaps=False
)

Why didn't we find anything?  Because we were looking for an exact match on the list of FEC IDs - by default, `get_if_exists` treats arrays just like any other value. But if we pass `allow_list_overlaps=True`, we can tell django_pewtils to not only search for existing records that overlap with our list, but also to update the existing record with the _union_ of the lists rather than overwrite what's already there. (Note: this only works with databases that support array fields, aka Postgres)

In [32]:
Politician.objects.get_if_exists(
    {"icpsr_id": new_record["icpsr_id"], "fec_ids": new_record["fec_ids"]},
    match_any=True, search_nulls=False, empty_lists_are_null=True, allow_list_overlaps=True
)

<Politician: Dwayne 'The Rock' Johnson>

In [33]:
mr_rock = Politician.objects.create_or_update(
    {"icpsr_id": new_record["icpsr_id"], "fec_ids": new_record["fec_ids"]},
    new_record,
    match_any=True, search_nulls=False, empty_lists_are_null=True, allow_list_overlaps=True,
    save_nulls=False, only_update_existing_nulls=False, return_object=True
)

In [34]:
mr_rock.json(exclude_nulls=True)

{'id': 64716,
 'first_name': 'Dwayne',
 'last_name': 'Johnson',
 'nickname': 'The Rock',
 'has_press_release_scraper': False,
 'bioguide_id': 'J99999',
 'fec_ids': ['P99999', 'S99999'],
 'icpsr_id': '12345',
 'facebook_ids': [],
 'old_facebook_ids': [],
 'twitter_ids': [],
 'old_twitter_ids': [],
 'instagram_ids': ['1234567890'],
 'old_instagram_ids': [],
 'capitol_words_speech_backfill': False}

Boom - it found our existing record with an overlapping FEC ID, updated it with the new ICPSR ID, update the FEC IDs to the union, and also avoided overwriting the existing nickname.

Let's try saving one more record, but this time we don't have any unique identifiers that overlap with our existing data.  And there's a typo in the data.  Great.  This is going to cause some problems.

In [35]:
new_record = {
    "opensecrets_id": "12345",
    "instagram_ids": ["0987654321"],
    "nickname": "Teh Rock",
    "last_name": "Johnson"
}

In [36]:
also_mr_rock = Politician.objects.create_or_update(
    {"opensecrets_id": new_record["opensecrets_id"]},
    new_record,
    match_any=True, search_nulls=False, empty_lists_are_null=True, allow_list_overlaps=True,
    save_nulls=False, only_update_existing_nulls=False, return_object=True
)

Now we've unwittingly created two different records for Mr. Rock, despite our best efforts to leverage all of the overlapping unique identifiers from our various data sources.

In [37]:
mr_rock.json(exclude_nulls=True)

{'id': 64716,
 'first_name': 'Dwayne',
 'last_name': 'Johnson',
 'nickname': 'The Rock',
 'has_press_release_scraper': False,
 'bioguide_id': 'J99999',
 'fec_ids': ['P99999', 'S99999'],
 'icpsr_id': '12345',
 'facebook_ids': [],
 'old_facebook_ids': [],
 'twitter_ids': [],
 'old_twitter_ids': [],
 'instagram_ids': ['1234567890'],
 'old_instagram_ids': [],
 'capitol_words_speech_backfill': False}

In [38]:
also_mr_rock.json(exclude_nulls=True)

{'id': 64719,
 'last_name': 'Johnson',
 'nickname': 'Teh Rock',
 'has_press_release_scraper': False,
 'fec_ids': [],
 'opensecrets_id': '12345',
 'facebook_ids': [],
 'old_facebook_ids': [],
 'twitter_ids': [],
 'old_twitter_ids': [],
 'instagram_ids': ['0987654321'],
 'old_instagram_ids': [],
 'capitol_words_speech_backfill': False}

What can we do about this?  We know about each copy right now - so in this case, we could manually write some code to resolve the two records and delete one of them - but in many cases, we aren't even going to be _aware_ that a duplicate got created.  So our first challenge is: how do we check a massive database for possible duplicates, if we've already checked all of the obvious unique indicators?  And our second challenge is: if we run into one of these duplicates, how do we resolve them without having to do it manually every single time?

So let's start with our original Mr. Rock, and see if we can find his clone.  In this case, the best shot we have at doing this is to look for other Politician records with similar names.  We have a few different fields that might be useful - first name, last name, and nickname.  Fortunately, Django Pewtils' `BasicExtendedModel` and `BasicExtendedManager` offer a variety of text similarity search functions.

In [39]:
mr_rock

<Politician: Dwayne 'The Rock' Johnson>

In [40]:
mr_rock.similar_by_fuzzy_ratios(['first_name', 'nickname', 'last_name'], min_ratio=.9)[:3]

[{'pk': 64719,
  'first_name': '',
  'nickname': 'Teh Rock',
  'last_name': 'Johnson',
  'fuzzy_ratio': 80.0},
 {'pk': 55405,
  'first_name': 'Dan',
  'nickname': '',
  'last_name': 'Johnson',
  'fuzzy_ratio': 68.57142857142857},
 {'pk': 54070,
  'first_name': 'Daniel',
  'nickname': '',
  'last_name': 'Johnson',
  'fuzzy_ratio': 68.42105263157895}]

In [41]:
mr_rock.similar_by_levenshtein_differences(['first_name', 'nickname', 'last_name'], max_difference=.5)[:3]

<PoliticianManager [{'difference': 0.470588235294118, 'pk': 64719, 'first_name': '', 'nickname': 'Teh Rock', 'last_name': 'Johnson'}]>

In [42]:
mr_rock.similar_by_tfidf_similarity(['first_name', 'nickname', 'last_name'], min_similarity=.5)[:3]

[{'pk': 64719,
  'first_name': '',
  'nickname': 'Teh Rock',
  'last_name': 'Johnson',
  'similarity': 0.5351794382465026}]

In [43]:
mr_rock.similar_by_trigram_similarity(['first_name', 'nickname', 'last_name'], min_similarity=.5)[:3]

<PoliticianManager [{'similarity': 0.518519, 'pk': 64719, 'first_name': '', 'nickname': 'Teh Rock', 'last_name': 'Johnson'}]>

Okay, so now we've found our duplicate.  Now what?  Well, Django Pewtils' `consolidate_objects` function has the ability to collapse duplicate records.  Not only will it let us easily merge our records together the way we'd like, it'll also resolve database relations intelligently.  Values that are null in one record but filled in the other will be filled in, many-to-many relationships and arrays will get merged into their unions, and if our records have any unique one-to-one relationships (e.g. each record has a unique "WikipediaPage" assigned to it, and those are duplicates as well), we can instruct `consolidate_objects` to cascade to those records and consolidate them as well.  All we need to do is specify which record we want to keep: the "source" is our duplicate, and we'll merge it into the "target", which is the record that we'll be keeping.

In [44]:
from django_pewtils import consolidate_objects

mr_rock = consolidate_objects(
    source=also_mr_rock,
    target=mr_rock,
    overwrite=False,  # False means that we'll prefer preserving the target's existing values if we encounter conflicts
    consolidate_related_uniques=False  # Unless we set this to True, the function will raise an error if there are conflicting relationships that can't be merged
)

In [45]:
mr_rock.json(exclude_nulls=True)

{'id': 64716,
 'first_name': 'Dwayne',
 'last_name': 'Johnson',
 'nickname': 'The Rock',
 'has_press_release_scraper': False,
 'bioguide_id': 'J99999',
 'fec_ids': ['S99999', 'P99999'],
 'opensecrets_id': '12345',
 'icpsr_id': '12345',
 'facebook_ids': [],
 'old_facebook_ids': [],
 'twitter_ids': [],
 'old_twitter_ids': [],
 'instagram_ids': ['0987654321', '1234567890'],
 'old_instagram_ids': [],
 'capitol_words_speech_backfill': False}

You know, it would have been nice to avoid all of this in the first place.  What if we could have written some additional checks when we first loaded in the duplicate Rock record, to search for existing politicians with similar names?  Our duplicate record had values for `nickname` and `last_name`, so what if we had scanned the database for matches using those?

In [46]:
search_text = "Teh Rock Johnson"  # What we had in our duplicate record

In [47]:
Politician.objects.fuzzy_ratios(['nickname', 'last_name'], search_text)[:3]

[{'pk': 64716,
  'nickname': 'The Rock',
  'last_name': 'Johnson',
  'fuzzy_ratio': 93.75},
 {'pk': 55400,
  'nickname': '',
  'last_name': 'Johnson',
  'fuzzy_ratio': 66.66666666666666},
 {'pk': 52587,
  'nickname': '',
  'last_name': 'Johnson',
  'fuzzy_ratio': 66.66666666666666}]

In [48]:
Politician.objects.fuzzy_ratio_best_match(['nickname', 'last_name'], search_text)

(<Politician: Dwayne 'The Rock' Johnson>, 93.75)

In [49]:
Politician.objects.levenshtein_differences(['nickname', 'last_name'], search_text)[:3]

<PoliticianManager [{'difference': 0.125, 'pk': 64716, 'nickname': 'The Rock', 'last_name': 'Johnson'}, {'difference': 0.533333333333333, 'pk': 48661, 'nickname': '', 'last_name': 'Mcneal Johnson'}, {'difference': 0.533333333333333, 'pk': 59287, 'nickname': '', 'last_name': 'Roldan-Johnson'}]>

In [50]:
Politician.objects.levenshtein_difference_best_match(['nickname', 'last_name'], search_text)

(<Politician: Dwayne 'The Rock' Johnson>, 0.125)

In [51]:
Politician.objects.tfidf_similarities(['nickname', 'last_name'], search_text)[:3]

[{'pk': 64716,
  'nickname': 'The Rock',
  'last_name': 'Johnson',
  'similarity': 0.8116603660917949},
 {'pk': 55651,
  'nickname': '',
  'last_name': 'Johnson',
  'similarity': 0.5205634156460611},
 {'pk': 49912,
  'nickname': '',
  'last_name': 'Johnson',
  'similarity': 0.5205634156460611}]

In [52]:
Politician.objects.tfidf_similarity_best_match(['nickname', 'last_name'], search_text)

(<Politician: Dwayne 'The Rock' Johnson>, 0.8116603660917949)

In [53]:
Politician.objects.trigram_similarities(['nickname', 'last_name'], search_text)[:3]

<PoliticianManager [{'similarity': 0.7, 'pk': 64716, 'nickname': 'The Rock', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 55400, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 52587, 'nickname': '', 'last_name': 'Johnson'}]>

In [54]:
Politician.objects.trigram_similarity_best_match(['nickname', 'last_name'], search_text)

(<Politician: Dwayne 'The Rock' Johnson>, 0.7)

Some of these searches can take quite a while to run and/or will eat up a lot of memory when you've got a large table.  Postgres also has a built-in search functionality that can more efficiently put the burden on your database.

In [55]:
Politician.objects.postgres_search(['nickname', 'last_name'], search_text)

<PoliticianManager [<Politician: Dwayne 'The Rock' Johnson>, <Politician: William C Kortz II>, <Politician: Robert Michael Clark>, <Politician: Abel Maldonado>, <Politician: Doyel Shamley>, <Politician: Jamie Moore>, <Politician: James A Barnett>, <Politician: Thomas Catalano>, <Politician: Abel Gebre Laeke>, <Politician: Andrew Michael Decker>, <Politician: L. Mack Van Allen>, <Politician: Henry W Meers Jr>, <Politician: Darrel Ervin Miller>, <Politician: Robert W. Tucker>, <Politician: Oreta Tufaga-Mapu Crichton>, <Politician: Phat Nguyen>, <Politician: Justin Sung-Sup Kim>, <Politician: Lei Sharsh-Davis>, <Politician: Alan J.K. Yim>, <Politician: Shirlene D. (Shirl) Ostrov>, '...(remaining elements truncated)...']>

Equivalent functions also exist on `BasicExtendedModel` so you can invoke them for specific records, too.

In [56]:
mr_rock.trigram_similarity(['nickname', 'last_name'], search_text)

0.7

In [57]:
mr_rock.similar_by_trigram_similarity(['nickname', 'last_name'], min_similarity=.4)

<PoliticianManager [{'similarity': 0.470588, 'pk': 52587, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 45952, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 46918, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 47003, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 48651, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 49075, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 63555, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 49386, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 49618, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 49836, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 49912, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 50417, 'nickname': '', 'last_name': 'Johnson'}, {'similarity': 0.470588, 'pk': 5

Okay, goodbye Mr. Rock.

In [58]:
mr_rock.delete()

(1, {'logos.Politician': 1})

Django Pewtils also has a variety of functions for inspecting records in your database.  Let's take a look at someone a bit more established than Mr. Rock.

In [59]:
bernie = Politician.objects.get(bioguide_id="S000033")

Let's see what we've got on Bernie.

In [60]:
bernie.related_objects()

{'personal_metrics': <QueryModelManager [<PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2016, dw_nominate1: -0.526>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2015, comfortable_with_samesex_marriage: 2.0>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2015, privatize_social_security: -2.0>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2019, dw_nominate1: -0.526>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2019, dw_nominate2: -0.371>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2020, dw_nominate1: -0.526>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2020, dw_nominate2: -0.371>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2009, dw_nominate2: -0.296999990940094>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2010, dw_nominate1: -0.50900000333786>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2010, dw_nominate2: -0.296999990940094>, <PoliticianPersonalMetric: Bernard 'Bernie' Sanders, 2011, dw_nominate2: -0

In [61]:
bernie.related_objects(counts=True)

{'personal_metrics': 47,
 'campaigns': 7,
 'staffers': 0,
 'relevant_news_articles': 96109,
 'press_releases': 3326,
 'ballotpedia_page': 1,
 'wikipedia_page': 1,
 'speeches': 2890,
 'twitter_profiles': 3,
 'incumbent_elections': 2,
 'elections_won': 11,
 'contributions_donated': 0,
 'contributions_received': 0,
 'facebook_pages': 2,
 'terms': 11,
 'chaired_committees': 0,
 'ranking_member_committees': 2,
 'committees': 0,
 'committee_memberships': 63,
 'caucuses': 1,
 'sponsored_bills': 235,
 'cosponsored_bills': 2638,
 'votes_for': 3320,
 'votes_against': 1918,
 'votes_abstained': 343,
 'hearings': 1059,
 'scrape_logs': 3,
 'webpages': 2193,
 'current_term': 1,
 'latest_term': 1,
 'party': 1,
 'commands': 21,
 'command_logs': 1108,
 'verifications': 0}

Looks like we've got a lot of data on Bernie.  What would happen if we deleted him?  Deleting a record in a database can cause a lot of unexpected cascade behavior if you aren't careful!  The `inspect_delete` function in Django Pewtils' `BasicExtendedModel` can help you make sure you're not going to do something you'll regret.  (You can also run this on queries too.)

In [62]:
bernie.inspect_delete(counts=True)

defaultdict(list,
            {logos.models.agents.Politician: 1,
             logos.models.agents.Politician_commands: 21,
             logos.models.agents.Politician_command_logs: 1108,
             logos.models.agents.PoliticianPersonalMetric: 47,
             logos.models.agents.PoliticianPersonalMetric_commands: 27,
             logos.models.agents.PoliticianPersonalMetric_command_logs: 167,
             logos.models.media.NewsArticle_relevant_politicians: 96109,
             logos.models.media.BallotpediaPage: 1,
             logos.models.media.WikipediaPage: 1,
             logos.models.media.WikipediaPage_commands: 1,
             logos.models.media.WikipediaPage_command_logs: 1,
             logos.models.government.CommitteeMembership: 63,
             logos.models.government.CommitteeMembership_commands: 63,
             logos.models.government.CommitteeMembership_command_logs: 63,
             logos.models.government.Caucus_members: 1,
             logos.models.government.Bi

So, let's maybe _not_ do that. 

Anyway, that's some of the handy stuff in Django Pewtils!

Let's make sure we deleted all of our Rocks, just in case:

In [63]:
for field, unique_id in [
    ("fec_ids", ["S99999", "P99999"]),
    ("fec_ids", ['P99999', 'H99999']),
    ("bioguide_id", "J99999"),
    ("icpsr_id", "12345"),
    ("opensecrets_id", "12345")
]:
    try: Politician.objects.get(**{field: unique_id}).delete()
    except Politician.DoesNotExist: pass